In [77]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/ironhack_final_project/')

Mounted at /content/drive


In [78]:
from google.colab import files
# uploaded = files.upload()

In [79]:
%matplotlib inline
import re
import numpy as np
import pandas as pd

In [80]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [81]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import pickle

#### SELL DATASET

### Exploring the data sets

Load the data file:

In [82]:
sell = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv_data/sell-bq-results.csv', encoding = "ISO-8859-1")

In [83]:
sell.head()

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,66837c9ddd9fc8958a007b0fa4b3316b8a3c4e2f,1/20/2018,sell,house,Colima,|México|Colima|,México,Colima,NaN,NaN,NaN,NaN,6560000.0,MXN,6543950.63,351957.76,NaN,600.0,NaN,10933.333330,NaN,NaN,NaN,https://www.properati.com.mx/19l0o_venta_casa_...,Nocnok ID: MX17-CW0049. Residencia demonÃ­aca ...,Casa en Venta en Altozano,https://thumbs4.properati.com/8/-tGAcFjNSFkx0N...
1,428900f7d6f0a341c9217b734a3a951daaef2fc7,1/19/2018,sell,house,Colima,|México|Colima|,México,Colima,NaN,NaN,NaN,NaN,315000.0,MXN,313251.15,16847.80,NaN,38.0,NaN,8289.473684,NaN,NaN,NaN,https://www.properati.com.mx/19fiz_venta_casa_...,"Nocnok ID: MX15-BG9213. CASA DE 2 RECAMARAS , ...",Casa en Venta en SANTA CLARA TRAPICHE SAN JOAQUIN,https://thumbs4.properati.com/1/1vowDywaTZmJSs...
2,83231c85a041485dfbd015824a1a43b9280c5472,1/19/2018,sell,house,Colima,|México|Colima|,México,Colima,NaN,NaN,NaN,NaN,395000.0,MXN,392806.87,21126.60,NaN,55.0,NaN,7181.818182,NaN,NaN,NaN,https://www.properati.com.mx/19fkn_venta_casa_...,Nocnok ID: MX15-BI0314. EXCELENTES CASAS EN EL...,Casa en Venta en LOMAS DE SANTA CRUZ,https://thumbs4.properati.com/4/WX_kFJhyz8cajs...
3,686b47efdb9d08a654bebc87cee0f96ab554f070,1/19/2018,sell,house,Colima,|México|Colima|,México,Colima,NaN,NaN,NaN,NaN,480000.0,MXN,477335.11,25672.84,NaN,75.0,NaN,6400.000000,NaN,NaN,NaN,https://www.properati.com.mx/19g76_venta_casa_...,Nocnok ID: MX16-BX3279. CASA CENTRICA DE 2 REC...,Casa en Venta en CENTRO CONOCIDO,https://thumbs4.properati.com/0/nZFoGCMyLZfkFW...
4,c684128f44727d870169a41c4fa1c47badefd3d3,1/19/2018,sell,house,Colima,|México|Colima|,México,Colima,NaN,NaN,NaN,NaN,550000.0,MXN,546946.38,29416.79,NaN,350.0,NaN,1571.428571,NaN,NaN,NaN,https://www.properati.com.mx/19gdc_venta_casa_...,Nocnok ID: MX16-CA5982. EXCELENTE CASA CENTRIC...,Casa en Venta en CENTRO QUESERIA,https://thumbs4.properati.com/3/PP3dAdkfxM7Ze1...


In [84]:
# Data types
sell.dtypes

id                             object
created_on                     object
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat_lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbn

In [85]:
# Rows and columns
sell.shape

(199514, 27)

In [86]:
sell.columns

Index(['id', 'created_on', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat_lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [87]:
# We proceed to eliminate the columns that do not interest
sell.drop(columns=['id', 'created_on', 'operation', 'place_name',
       'place_with_parent_names', 'country_name', 'geonames_id',
       'lat_lon', 'lat', 'lon', 'price', 'currency', 'price_aprox_usd', 
       'price_usd_per_m2','floor', 'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'], inplace=True)

In [88]:
sell.columns

Index(['property_type', 'state_name', 'price_aprox_local_currency',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_per_m2'],
      dtype='object')

In [89]:
# Handle Missing Values
sell.isnull().sum()

property_type                     0
state_name                        0
price_aprox_local_currency     1861
surface_total_in_m2           98913
surface_covered_in_m2         13833
price_per_m2                  23687
dtype: int64

In [90]:
# Delete Missing Values
sell.dropna(axis = 0, how = "any", inplace= True)

### Handle Categorical Data

In [91]:
# Show unique values
sell["state_name"].unique()

array(['Colima', 'Puebla', 'Hidalgo', 'Jalisco', 'Sinaloa', 'Tabasco',
       'Guerrero', 'Tlaxcala', 'Chihuahua', 'QuerÃ©taro', 'Tamaulipas',
       'Nuevo LeÃ³n', 'Durango', 'Morelos', 'Nayarit', 'Sonora',
       'YucatÃ¡n', 'Campeche', 'Chiapas', 'Guanajuato',
       'San Luis PotosÃ\xad', 'Ciudad de México', 'Estado de México',
       'Zacatecas', 'Oaxaca', 'Quintana Roo', 'Baja California Sur',
       'Baja California', 'Aguascalientes', 'Distrito Federal',
       'Coahuila de Zaragoza', 'Veracruz de Ignacio de la Llave'],
      dtype=object)

In [92]:
# Overwriting words with strange characters
sell["state_name"][sell["state_name"] == 'QuerÃ©taro'] = 'Querétaro' 
sell["state_name"][sell["state_name"] == 'Nuevo LeÃ³n'] = 'Nuevo León' 
sell["state_name"][sell["state_name"] == 'YucatÃ¡n'] = 'Yucatán' 
sell["state_name"][sell["state_name"] == 'San Luis PotosÃ\xad'] = 'San Luis Potosí' 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [93]:
# Show unique values
sell["state_name"].unique()

array(['Colima', 'Puebla', 'Hidalgo', 'Jalisco', 'Sinaloa', 'Tabasco',
       'Guerrero', 'Tlaxcala', 'Chihuahua', 'Querétaro', 'Tamaulipas',
       'Nuevo León', 'Durango', 'Morelos', 'Nayarit', 'Sonora', 'Yucatán',
       'Campeche', 'Chiapas', 'Guanajuato', 'San Luis Potosí',
       'Ciudad de México', 'Estado de México', 'Zacatecas', 'Oaxaca',
       'Quintana Roo', 'Baja California Sur', 'Baja California',
       'Aguascalientes', 'Distrito Federal', 'Coahuila de Zaragoza',
       'Veracruz de Ignacio de la Llave'], dtype=object)

### Evaluation of the level of collinearity of the data.


*  Correlation matrix using the numeric columns of the data set.



In [94]:
corrMatrix = sell.corr()
corrMatrix

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
price_aprox_local_currency,1.000000,0.106990,0.031237,0.065322
surface_total_in_m2,0.106990,1.000000,0.079260,0.000217
surface_covered_in_m2,0.031237,0.079260,1.000000,-0.001322
price_per_m2,0.065322,0.000217,-0.001322,1.000000


*  Heat map using seaborn to visualize which columns have high collinearity.

In [64]:
corrMatrix.style.background_gradient(cmap='coolwarm') 

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
price_aprox_local_currency,1.000000,0.106990,0.031237,0.065322
surface_total_in_m2,0.106990,1.000000,0.079260,0.000217
surface_covered_in_m2,0.031237,0.079260,1.000000,-0.001322
price_per_m2,0.065322,0.000217,-0.001322,1.000000


### Normalize data

Using the scikit-learn object MinMaxScaler.

In [96]:
sell = sell.reset_index(drop=True, inplace=False)

In [97]:
sell.head(2)

,property_type,state_name,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,house,Colima,7259472.12,536.0,433.0,16859.1224
1,house,Colima,7285619.27,536.0,433.0,16859.1224



Divide data into categories and numbers

In [98]:
# Subdataset with categorical variables
sell_str = sell[['property_type', 'state_name']]

In [99]:
sell_str.head(2)

,property_type,state_name
0,house,Colima
1,house,Colima


In [100]:
# Prepare data for normalization
sell_num = sell[['price_aprox_local_currency', 'surface_total_in_m2', 
               'surface_covered_in_m2', 'price_per_m2']]

In [101]:
sell_num.head(2)

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,7259472.12,536.0,433.0,16859.1224
1,7285619.27,536.0,433.0,16859.1224


In [108]:
# train the normalization
scaler = MinMaxScaler()
scaler = scaler.fit(sell_num)

In [112]:
# train the normalization for prediction
scaler_pred = MinMaxScaler()
sell_num_copy = sell_num[['surface_total_in_m2', 'surface_covered_in_m2', 'price_per_m2']]
scaler_pred = scaler_pred.fit(sell_num_copy)
with open("scaler", "wb") as f: 
    pickle.dump(scaler_pred, f)

In [113]:
# normalize the dataset and print the first 5 rows
normalized = scaler.transform(sell_num)

In [114]:
normalized

array([[0.00970481, 0.00268   , 0.00039273, 0.00024432],
       [0.00974021, 0.00268   , 0.00039273, 0.00024432],
       [0.00446928, 0.0025    , 0.00041   , 0.000109  ],
       ...,
       [0.04633426, 0.00416   , 0.00074455, 0.00060974],
       [0.03455089, 0.00396   , 0.00068546, 0.00049427],
       [0.00264473, 0.0006    , 0.00014455, 0.00018567]])

In [115]:
normalized.shape

(77743, 4)

In [116]:
column_names = ['price_aprox_local_currency', 'surface_total_in_m2', 
               'surface_covered_in_m2', 'price_per_m2']
sell_num_norma = pd.DataFrame(normalized, columns=column_names)

In [118]:
sell_num_norma.head(2)

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,0.009705,0.00268,0.000393,0.000244
1,0.009740,0.00268,0.000393,0.000244


In [119]:
new_sell = pd.concat([sell_str,sell_num_norma], axis=1)

In [121]:
new_sell.head(2)

,property_type,state_name,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,house,Colima,0.009705,0.00268,0.000393,0.000244
1,house,Colima,0.009740,0.00268,0.000393,0.000244


In [175]:
# # Exportar dataset 
# new_sell.to_csv("drive/MyDrive/Colab Notebooks/ironhack_final_project/sell-bq-results_clean.csv", index = False, encoding="ISO-8859-1")

### RENT DATASET

### 1. Exploring the data sets

Load the data file:

In [122]:
rent = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv_data/rent-bq-results.csv', encoding = "ISO-8859-1")

In [123]:
rent.head()

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,add247ef3904234e5e1cb7c552635d5097783ea6,2017-10-04,rent,house,Colima,|MÃ©xico|Colima|,MÃ©xico,Colima,NaN,"19.336132,-103.601901",19.336132,-103.601901,10000.0,MXN,9975.52,536.52,NaN,400.0,NaN,25.000000,NaN,NaN,NaN,https://www.properati.com.mx/1519b_venta_casa_...,Nocnok ID: MX17-DN1738. Hermosa residencia ubi...,Casa en Renta en San Rafael,https://thumbs4.properati.com/6/eMZPHOCwrvbRz0...
1,7a384eea7e7573eed7c1cac79154d2ec45ee6e1a,2017-10-30,rent,house,Colima,|MÃ©xico|Colima|,MÃ©xico,Colima,NaN,"23.634501,-102.552788",23.634501,-102.552788,7500.0,MXN,7545.97,405.85,NaN,90.0,NaN,83.333333,NaN,NaN,NaN,https://www.properati.com.mx/16pmc_renta_casa_...,"Nocnok ID: MX17-DP8120. 1 nivel, 3 recÃ¡maras ...",Casa en Renta en El Trapiche,https://thumbs4.properati.com/3/exf3tjTxhX4sZv...
2,3eca6db7fb229d0edd3f7680ad7c8ba9ba1e84ae,2017-08-12,rent,apartment,Colima,|MÃ©xico|Colima|,MÃ©xico,Colima,NaN,"19.2568989,-103.6974246",19.256899,-103.697425,3500.0,MXN,3493.07,187.87,40.0,NaN,4.69675,NaN,NaN,1.0,NaN,https://www.properati.com.mx/126ko_renta_depar...,"Departamento amueblado, televisiÃ³n con dish e...",Departamento renta,https://thumbs4.properati.com/1/JsG6qii2on_3e4...
3,27d5d6ff4ca87256709ff012a6c5441f2edd90bf,2018-02-01,rent,house,Sonora,|MÃ©xico|Sonora|,MÃ©xico,Sonora,NaN,"29.086493,-111.037102",29.086493,-111.037102,25000.0,MXN,24861.07,1337.12,0.0,0.0,NaN,NaN,NaN,NaN,NaN,https://www.properati.com.mx/1c6ye_renta_casa_...,"Preciosa Casa en RENTA en REAL DE SEVILLA, Equ...","Preciosa Casa en RENTA en REAL DE SEVILLA, Equ...",https://thumbs4.properati.com/7/z96reMgsO2DnFJ...
4,9c6e6a1739d971e4558579272d57902e8c447101,2017-12-06,rent,house,Sonora,|MÃ©xico|Sonora|,MÃ©xico,Sonora,NaN,"29.086493,-111.037102",29.086493,-111.037102,25000.0,MXN,24950.69,1341.94,0.0,0.0,NaN,NaN,NaN,NaN,NaN,https://www.properati.com.mx/18fho_renta_casa_...,"Preciosa Casa en RENTA en REAL DE SEVILLA, Equ...","Preciosa Casa en RENTA en REAL DE SEVILLA, Equ...",https://thumbs4.properati.com/7/z96reMgsO2DnFJ...


In [124]:
# Data types
rent.dtypes

id                             object
created_on                     object
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat_lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbn

In [127]:
# Rows and columns
rent.shape

(46566, 27)

In [128]:
rent.columns

Index(['id', 'created_on', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat_lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [129]:
# We proceed to eliminate the columns that do not interest
rent.drop(columns=['id', 'created_on', 'operation', 'place_name',
       'place_with_parent_names', 'country_name', 'geonames_id',
       'lat_lon', 'lat', 'lon', 'price', 'currency', 'price_aprox_usd', 
       'floor', 'price_usd_per_m2', 'rooms', 'expenses', 
       'properati_url', 'description', 'title', 'image_thumbnail'], inplace=True)

In [130]:
rent.columns

Index(['property_type', 'state_name', 'price_aprox_local_currency',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_per_m2'],
      dtype='object')

In [131]:
# Handle Missing Values
rent.isnull().sum()

property_type                     0
state_name                        0
price_aprox_local_currency     1545
surface_total_in_m2           16369
surface_covered_in_m2          3723
price_per_m2                  11549
dtype: int64

In [132]:
# Delete Missing Values
rent.dropna(axis = 0, how = "any", inplace= True)

In [133]:
rent.shape

(19165, 6)

### Handle Categorical Data

In [134]:
# Show unique values
rent["state_name"].unique()

array(['Hidalgo', 'Morelos', 'Tabasco', 'Guerrero', 'Chihuahua',
       'Tamaulipas', 'Colima', 'Nayarit', 'Puebla', 'Sonora', 'YucatÃ¡n',
       'Campeche', 'Durango', 'Jalisco', 'Sinaloa', 'Guanajuato',
       'Tlaxcala', 'Zacatecas', 'Oaxaca', 'Quintana Roo', 'Nuevo LeÃ³n',
       'QuerÃ©taro', 'San Luis PotosÃ\xad', 'Aguascalientes',
       'Baja California', 'Estado de MÃ©xico', 'Chiapas',
       'Baja California Sur', 'Distrito Federal', 'Coahuila de Zaragoza',
       'Veracruz de Ignacio de la Llave'], dtype=object)

In [135]:
# Overwriting words with strange characters
rent["state_name"][rent["state_name"] == 'QuerÃ©taro'] = 'Querétaro' 
rent["state_name"][rent["state_name"] == 'Nuevo LeÃ³n'] = 'Nuevo León' 
rent["state_name"][rent["state_name"] == 'YucatÃ¡n'] = 'Yucatán' 
rent["state_name"][rent["state_name"] == 'San Luis PotosÃ\xad'] = 'San Luis Potosí'
rent["state_name"][rent["state_name"] == 'Estado de MÃ©xico'] = 'Estado de México' 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [136]:
# Show unique values
rent["state_name"].unique()

array(['Hidalgo', 'Morelos', 'Tabasco', 'Guerrero', 'Chihuahua',
       'Tamaulipas', 'Colima', 'Nayarit', 'Puebla', 'Sonora', 'Yucatán',
       'Campeche', 'Durango', 'Jalisco', 'Sinaloa', 'Guanajuato',
       'Tlaxcala', 'Zacatecas', 'Oaxaca', 'Quintana Roo', 'Nuevo León',
       'Querétaro', 'San Luis Potosí', 'Aguascalientes',
       'Baja California', 'Estado de México', 'Chiapas',
       'Baja California Sur', 'Distrito Federal', 'Coahuila de Zaragoza',
       'Veracruz de Ignacio de la Llave'], dtype=object)

### Evaluation of the level of collinearity of the data.


*  Correlation matrix using the numeric columns of the data set.



In [137]:
corrMatrix = rent.corr()
corrMatrix

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
price_aprox_local_currency,1.000000,0.066735,0.357933,0.113337
surface_total_in_m2,0.066735,1.000000,0.164973,-0.009316
surface_covered_in_m2,0.357933,0.164973,1.000000,-0.049216
price_per_m2,0.113337,-0.009316,-0.049216,1.000000


*  Heat map using seaborn to visualize which columns have high collinearity.

In [138]:
corrMatrix.style.background_gradient(cmap='coolwarm') 

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
price_aprox_local_currency,1.000000,0.066735,0.357933,0.113337
surface_total_in_m2,0.066735,1.000000,0.164973,-0.009316
surface_covered_in_m2,0.357933,0.164973,1.000000,-0.049216
price_per_m2,0.113337,-0.009316,-0.049216,1.000000


### Normalize data

Using the scikit-learn object MinMaxScaler.

In [142]:
rent = rent.reset_index(drop=True, inplace=False)


Divide data into categories and numbers

In [143]:
# Subdataset with categorical variables
rent_str = rent[['property_type', 'state_name']]

In [144]:
rent_str.head(2)

,property_type,state_name
0,house,Hidalgo
1,house,Hidalgo


In [145]:
rent_str.shape

(19165, 2)

In [146]:
# Prepare data for normalization
rent_num = rent[['price_aprox_local_currency', 'surface_total_in_m2', 
               'surface_covered_in_m2', 'price_per_m2']]

In [147]:
rent_num.head(2)

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,10978.42,150.0,238.0,46.218487
1,6487.28,105.0,80.0,81.250000


In [148]:
# train the normalization
scaler = MinMaxScaler()
scaler = scaler.fit(rent_num)

In [149]:
print(scaler.data_max_)

[1785475.75  140000.     15000.     48000.  ]


In [150]:
# normalize the dataset and print the first 5 rows
normalized = scaler.transform(rent_num)

In [151]:
print(normalized)

[[0.00563099 0.00107143 0.01580105 0.0009568 ]
 [0.00311431 0.00075    0.00526702 0.00168663]
 [0.00507171 0.00187857 0.009934   0.00138281]
 ...
 [0.01457916 0.00142857 0.01726782 0.00215738]
 [0.02065473 0.00387143 0.03160211 0.00166059]
 [0.020731   0.00387143 0.03160211 0.00166059]]


In [154]:
column_names = ['price_aprox_local_currency', 'surface_total_in_m2', 
               'surface_covered_in_m2', 'price_per_m2']
rent_num_norma = pd.DataFrame(normalized, columns=column_names)

In [155]:
rent_num_norma.head()

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,0.005631,0.001071,0.015801,0.000957
1,0.003114,0.000750,0.005267,0.001687
2,0.005072,0.001879,0.009934,0.001383
3,0.005052,0.001879,0.009934,0.001383
4,0.000315,0.000857,0.007934,0.000254


In [156]:
new_rent = pd.concat([rent_str,rent_num_norma], axis=1)

In [157]:
new_rent.head()

,property_type,state_name,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2
0,house,Hidalgo,0.005631,0.001071,0.015801,0.000957
1,house,Hidalgo,0.003114,0.000750,0.005267,0.001687
2,house,Hidalgo,0.005072,0.001879,0.009934,0.001383
3,house,Hidalgo,0.005052,0.001879,0.009934,0.001383
4,house,Hidalgo,0.000315,0.000857,0.007934,0.000254


In [176]:
# # Exportar dataset 
# new_rent.to_csv("drive/MyDrive/Colab Notebooks/ironhack_final_project/rent-bq-results_clean.csv", index = False, encoding="ISO-8859-1")


## Prediction Model Trainings - Sell Data

### Modeling, Prediction, and Evaluation

In [159]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

df_enc = pd.DataFrame(enc.fit_transform(new_sell[['property_type', 'state_name']]).toarray())
print(df_enc.head())

with open("encoder", "wb") as f: 
    pickle.dump(enc, f)

    0    1    2    3    4    5    6   ...   29   30   31   32   33   34   35
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[5 rows x 36 columns]


In [160]:
# testing instance of one-hot-encoder
with open("encoder", "rb") as f: 
    enc = pickle.load(f)

my_test = {'property_type': 'house', 'state_name': 'Colima'}
my_test_df = pd.DataFrame(my_test, index=[0])
print(my_test_df)
coded = enc.transform(my_test_df[['property_type', 'state_name']]).toarray()
print(coded)
my_test_df = pd.DataFrame(coded)
print(my_test_df)

  property_type state_name
0         house     Colima
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
    0    1    2    3    4    5    6   ...   29   30   31   32   33   34   35
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[1 rows x 36 columns]


In [161]:
# dataset mount with modified variables for training
sell_4_train = pd.concat([sell_num_norma, df_enc], axis=1)
sell_4_train.head(2)

,price_aprox_local_currency,surface_total_in_m2,surface_covered_in_m2,price_per_m2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0.009705,0.00268,0.000393,0.000244,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.009740,0.00268,0.000393,0.000244,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
X = sell_4_train.drop(columns=["price_aprox_local_currency"],axis=1)
y = sell_4_train["price_aprox_local_currency"]

In [163]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=10)

In [171]:
# We store all the models that we are going to train in a dictionary
models = { "ridge": Ridge(),
          "lasso": Lasso(),
          "sgd": SGDRegressor(),
          "gradient": GradientBoostingRegressor(),
          'DecisionTree': DecisionTreeRegressor(max_depth=30, random_state=42),
          'RandomForest': RandomForestRegressor(),
          'MLP': MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes=(30, 100, 80,10))
} 

In [172]:
# We iterate on the models to train them
for name, model in models.items():
    print(f"Training model ---> {name}")
    model.fit(X_train, y_train)
    filename = name + '.pkl'
    pickle.dump(model, open(filename, 'wb'))
    print(f"OK")

Training model ---> ridge
OK
Training model ---> lasso
OK
Training model ---> sgd
OK
Training model ---> gradient
OK
Training model ---> DecisionTree
OK
Training model ---> RandomForest
OK
Training model ---> MLP
OK


In [173]:
# We iterate on the models to load them
for name, model in models.items():
    print(f"Loading model ---> {name}")
    filename = name + '.pkl'
    models[name] = pickle.load(open(filename, 'rb'))
    print(f"OK")

Loading model ---> ridge
OK
Loading model ---> lasso
OK
Loading model ---> sgd
OK
Loading model ---> gradient
OK
Loading model ---> DecisionTree
OK
Loading model ---> RandomForest
OK
Loading model ---> MLP
OK


In [174]:
# Get predictions of each algorithm and its metrics
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"--------{name}--------")
    print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))
    print("MSE: ", metrics.mean_squared_error(y_test, y_pred))
    print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
    print("R2: ", metrics.r2_score(y_test,y_pred))
    print("\n")

--------ridge--------
MAE:  0.005844817355244471
MSE:  0.00020221256950435796
RMSE:  0.01422014660628919
R2:  0.08895853334777648


--------lasso--------
MAE:  0.0065483635021560665
MSE:  0.00022197725433344568
RMSE:  0.014898901111607046
R2:  -8.858918642862257e-05


--------sgd--------
MAE:  0.005781023594772681
MSE:  0.00020673568788105693
RMSE:  0.014378306154796431
R2:  0.06858023337438801


--------gradient--------
MAE:  0.00202856813771708
MSE:  7.589895130182835e-05
RMSE:  0.008712000419067273
R2:  0.6580475087138775


--------DecisionTree--------
MAE:  0.0007060036117000818
MSE:  4.2596443116431924e-05
RMSE:  0.006526595063004286
R2:  0.808087469013019


--------RandomForest--------
MAE:  0.0006556493268400462
MSE:  3.7585532132256655e-05
RMSE:  0.006130704048660044
R2:  0.8306634528080737


--------MLP--------
MAE:  0.005846150030102872
MSE:  0.0001961426233775163
RMSE:  0.014005092765759043
R2:  0.11630585718354147


